# Creating a 3D Model

We use Build123d to create a #D model of the race track. We export the 3D model as a stl file for lagter import into Gazebo.

We also export a 2D projection as svg file. We use Inkskape to convert the svg file to a pgn file and then Image Magic to convert the png file to a pgm file. Adding some meta data will provide us with a map thatb we can use in Nav2

In [1]:
from jupyter_cadquery import show, open_viewer, set_defaults
import cadquery as cq
from build123d import *
cv = open_viewer("Build123d", cad_width=770, glass=True)
set_defaults(edge_accuracy=0.0001)

Overwriting auto display for cadquery Workplane and Shape


In [2]:
class SideWall():
    
    def __init__(self, whichWall):
        ext_r = 150
        int_r = 50
        lenght = 1000
        width = 551

        len1 = 700
        arc1 = 180
        len2 = 150
        arc2 = -90
        len3 = 50
        arc3 = 360 - 168.2
        len3 = 50
        len4 = 58.78
        arc4 = -122.2
        len5 = 103.05
        arc5 = 360 - 159.6

        wall_hight = 20
        wall_thick = 2
        
        if whichWall == 'ext':
            r1 = ext_r
            r2 = int_r
        else:
            r1 = int_r
            r2 = ext_r

        with BuildPart() as self.p:
            with BuildLine() as self.l:
                base_line = Line((0,0),(len1,0))
                rigth_curve = JernArc(start=base_line @ 1, tangent=base_line % 1, radius=r1, arc_size=arc1)
                line2 = PolarLine(rigth_curve @ 1, len2, direction=rigth_curve % 1)
                rigth_upper_curve = JernArc(start=line2 @ 1, tangent=line2 % 1, radius=r2, arc_size=arc2)
                line3 = PolarLine(rigth_upper_curve @ 1, len3, direction=rigth_upper_curve%1)
                upper_curve = JernArc(start=line3 @ 1, tangent=line3 % 1, radius=r1, arc_size=arc3)
                line4 = PolarLine(upper_curve @ 1, len4, direction=upper_curve%1)
                rigth_upper_curve = JernArc(start=line4 @ 1, tangent=line4%1, radius=r2, arc_size=arc4)
                line5 = PolarLine(rigth_upper_curve @ 1, len5, direction=rigth_upper_curve%1)
                left_curve = JernArc(start=line5 @ 1, tangent=line5%1, radius=r1, arc_size=arc5)
            with BuildSketch(Plane.XZ) as s:
                Rectangle(wall_thick, wall_hight, align=(Align.MAX, Align.MAX))
            sweep(is_frenet=True)

In [3]:
p_int = SideWall('int')
p_ext = SideWall('ext')

In [4]:
p_int.p.part.label = "internal wall"
p_int.p.part.location = Location((0, 100, 0))
p_ext.p.part.label = "external wall"

piste_assembly = Compound(label="piste", children=[p_int.p.part, p_ext.p.part])

In [5]:
print(piste_assembly.show_topology())


piste             Compound at 0x1ac453450, Location(p=(0.00, 0.00, 0.00), o=(-0.00, 0.00, -0.00))
├── internal wall Part     at 0x11330dc50, Location(p=(0.00, 100.00, 0.00), o=(-0.00, 0.00, -0.00))
└── external wall Part     at 0x1b05f3d90, Location(p=(0.00, 0.00, 0.00), o=(-0.00, 0.00, -0.00))



In [6]:
piste_assembly

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.18s


In [7]:
piste_assembly.export_stl('piste.stl')

True

In [8]:
piste_assembly.export_svg('piste.svg', (0, 0, 1000000), (0, 1, 0), 
                          svg_opts={"pixel_scale": 1, "margin_left": 0, "margin_top": 0,"show_axes": False, "show_hidden": False})

In [9]:
import xml.etree.ElementTree as ET
tree = ET.parse('piste.svg')
root = tree.getroot()
dim = root.attrib

In [10]:
import os
cmd = "inkscape -w %s -h %s piste.svg -o piste.png" % (dim['width'], dim['height'])
os.system(cmd)

0

In [11]:
!convert piste.png -flatten piste.pgm

In [12]:
metadata = {
    "image": "piste.pgm",
    "resolution": 0.01,
    "origin": [0.0, 0.0, 0.0],
    "occupied_thresh": 0.65,
    "free_thresh": 0.196,
    "negate": 0
}

In [13]:
metadata

{'image': 'piste.pgm',
 'resolution': 0.01,
 'origin': [0.0, 0.0, 0.0],
 'occupied_thresh': 0.65,
 'free_thresh': 0.196,
 'negate': 0}

In [ ]:
import yaml
with open("piste.yaml", "w") as fh:
    yaml.dump(metadata, fh)